<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/NN-online/blob/main/practice0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice 1

In this  exercise we __modify__ the feedforward neural network we used  to recognize handwrittne digits (the MNIST data).  The modification involves 
1. the activation function for the intermediate layers. Instead of hardcoding them as we did before we supply them as parameters. 

For example if we have 3 layers with sigmoid for the intermediate and softmax for the last layer we supply the activations as a list of pair of functions. Each pair refers to the activation and its derivative.
 
```
activations=[(sigmoid,dsigmoid),(sigmoid,dsigmoid),(softmax,None)]

```
1. We use another dataset: "Iris" dataset



### Packages

The data set is very small so there is no need for  __cupy__.

In [1]:
import numpy as np
import random
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer



### Download data from Kaggle

In [2]:
from google.colab import files
file=files.upload()
!mkdir /root/.kaggle
!mv kaggle.json  /root/.kaggle
!kaggle datasets download -d uciml/iris
!unzip iris.zip

Saving kaggle.json to kaggle.json
  0% 0.00/3.60k [00:00<?, ?B/s]
100% 3.60k/3.60k [00:00<00:00, 3.21MB/s]
Archive:  iris.zip
  inflating: Iris.csv                
  inflating: database.sqlite         


### Read the data using pandas

Read the data 

In [6]:
df=pd.read_csv("Iris.csv")


Inspect the data

In [7]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


we drop the first column which contains a sequential id. 

In [8]:
df=df.drop('Id',axis=1)


We also shuffle the data using the sklearn shuffle

In [9]:
df = shuffle(df)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
142,5.8,2.7,5.1,1.9,Iris-virginica
89,5.5,2.5,4.0,1.3,Iris-versicolor
57,4.9,2.4,3.3,1.0,Iris-versicolor
106,4.9,2.5,4.5,1.7,Iris-virginica
72,6.3,2.5,4.9,1.5,Iris-versicolor
...,...,...,...,...,...
96,5.7,2.9,4.2,1.3,Iris-versicolor
80,5.5,2.4,3.8,1.1,Iris-versicolor
21,5.1,3.7,1.5,0.4,Iris-setosa
16,5.4,3.9,1.3,0.4,Iris-setosa


We use sklearn LabelBinarizer to convert the names of the species to one_hot encoding

In [10]:
species_lb = LabelBinarizer()
labels = species_lb.fit_transform(df.Species.values)

Inspect the one hot encoding of the labels

In [11]:
labels[0:10]

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])

Retrieve the features

In [12]:
features = df[df.columns[0:4]].values

Split data into train and test

In [ ]:
x_train=features[0:120]
x_test=features[120:150]
y_train=labels[0:120]
y_test=labels[120:150]

### Activation Functions

Typically in such a setting the activation function for the inner layers could be sigmoid, relu,... and the activation for the last layer would be softmax which is a generalization of the sigmoid for multiclass inputs. Let $C$ be the set of classes and $c_1\in C$ a given class the probability of $c_1$ becomes (where $z_i$ is the logit corrsponding to class $i$)
\begin{align*}
softmax=\frac{e^{z_1}}{\sum_i e^{z_i}}
\end{align*}

In [ ]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x),axis=-1).reshape(x.shape[0],1)


## Derivative of the sigmoid

The derivative of the sigmoid with respect to its parameter is easily obtained
\begin{align*}
\frac{\partial\sigma(z)}{\partial z}&=\frac{1}{\partial z}\frac{1}{1+e^{-z}}\\
                                    &=\frac{e^{-z}}{(1+e^{-z})^2}\\
                                    &=\frac{1}{1+e^{-z}}\cdot \frac{e^{-z}}{1+e^{-z}}\\
                                    &=\sigma(z)(1-\sigma(z))
\end{align*}


In [ ]:
#derivative of the sigmoid
def dsigmoid(x):
  return sigmoid(x)*(1-sigmoid(x))

The usual cross Entropy cost. Unlike the evaluate function both the "true" labels and the output of our model are in one-hot encoding.
__NOTE__: the call to forward_propagation was changed to take an extra parameter: activations

In [ ]:
def compute_cost(X,Y,b,w,activations):
    
    m = Y.shape[0] # number of example

    # Compute the cross-entropy cost
    As,_=forward_propagation(X,b,w,activations)
    # recall that As contains the "output" of all layers
    # including the input As[0] and the final output As[-1]
    output=As[-1]
    logprobs = np.log(output)*Y
    cost = -np.sum(logprobs)/m

    count=0
    for i in range(output.shape[0]):
        if (np.argmax(output[i,:])==np.argmax(Y[i,:])):
            count=count+1

    
    cost = np.squeeze(cost)     # makes sure cost is the dimension we expect. 
                                # E.g., turns [[17]] into 17 
    
    return cost,count


The weights are initialized randomly and the biases are initially set to zero

In [ ]:
def initialize_parameters(width):    
    weights=[]
    biases=[]
    for i in range(len(width)-1):
        w=np.random.randn(width[i],width[i+1])
        b=np.zeros((width[i+1],))
        weights.append(w)
        biases.append(b)

    return biases,weights


### __NEEDS CHANGE__: forward_propagation
Forward propagation over all the layers but also retain the intermediate results.
1. takes an extra parameter: list of activation functions
1. use the proper activation for each layer
1. returns logits Z in addition to A

In [ ]:
def forward_propagation(X,biases,weights,activations):
    a=X
    As=[X]
    Z=[X]
    para=zip(weights,biases,activations)
    for i,(w,b,act) in enumerate(para):
       z=np.dot(a,w)+b
       ### REPLACE the calls with activations function from
       ### activation list. Remember it is a list of tuples(act,derivative)
       if i==len(biases)-1:
         a=softmax(z)
       else:
         a=sigmoid(z)
       #########################################################
       As.append(a)
       Z.append(z)
    return As,Z

## Backpropagation

Note how the gradient of a given layer depends on the gradient of the __next__ layer. This means that when computing the gradients we start from the __last__ layer and move __backwards__.
\begin{align*}
\Delta^l_{mn}&=\sum_k \Delta^{l+1}_{mk}W^l_{nk}\theta^l_{mn}\\
dW_{ij}&=\frac{1}{m}\sum_m\Delta^{l+1}_{mj}A^l_{mi}\\
dB_{j}&=\frac{1}{m}\sum_m\Delta^{l+1}_{mj}
\end{align*}

Or in matrix notation
\begin{align*}
\Delta^l&=\Delta^{l+1}\cdot (W^l)^T\theta^l\\
dW^l&=\frac{1}{m}(A^l)^T\cdot \Delta^{l+1}
\end{align*}

### __NEEDS CHANGE__: Backpropagation

theta depends on the activation of the layer 

In [ ]:
def backward_propagation(X,Y,biases,weights,activations):
    
    As,Z=forward_propagation(X,biases,weights,activations)
    m = X.shape[0]
    
    nlayers=len(biases)
    # Loss of the last layer
    dz=As[nlayers]-Y

    gradb=[]
    gradw=[]
    #iterator backwards from the last layer
    for i in range(nlayers-1,-1,-1):
        db=np.sum(dz,axis=0,keepdims=True)/m
        dw=np.dot(As[i].T,dz)/m
        # prepare the computation for the
        # NEXT iteration. If i=0 there is no
        # next iteration.
        if i!=0:
        ### REPLACE with the derivative of the activation
          #theta=activations[i-1][1](Z[i])
          theta=As[i]*(1-As[i])
        #################################################
          dz=np.dot(dz,weights[i].T)*theta
        gradb.insert(0,db)
        gradw.insert(0,dw)
    
    return gradb,gradw

In [ ]:
def update_parameters(biases,weights, gradb,gradw, learning_rate):
    for i in range(len(biases)):
        weights[i]=weights[i]-learning_rate*gradw[i]
        biases[i]=biases[i]-learning_rate*gradb[i]

    return biases,weights

### Stochastic Gradient Descent

We deal with the input data in __random batches__. We select a set of random indices and take a slice from X and Y

In [ ]:
def GD(X, Y, test_data,width,activations,batch_size,num_iterations, learning_rate,print_cost=False):

    biases,weights=initialize_parameters(width)
    
    for i in range(0, num_iterations):
        cost,count = compute_cost(X,Y,biases,weights,activations)
        for k in range(0,X.shape[0],batch_size):
            ## We CANNOT use np.random.shuffle because
            ## it would randomize x and y DIFFERENTLY
            ## instead get a set of random  indices
            ## and use the SAME indices for x and y
            idx=[random.randint(0,Y.shape[0]-1) for s in range(batch_size)]
            yb=Y[idx,:]
            xb=X[idx,:]
            gradb,gradw=backward_propagation(xb,yb,biases,weights,activations)
            biases,weights=update_parameters(biases,weights,gradb,gradw,learning_rate)    

        if i%20 ==0 : 
            #count_test=evaluate(test_data,test_labels,biases,weights)
            count=count/Y.shape[0]
            print ("Epoch {}: cost={:.2f},train accuracy={:.2f}".format(i,cost,count))
                
    return biases,weights

#### __NEEDS CHANGE__ : run SGD

In [ ]:
##Determine the shape of the input and output
##
n_x=None
n_y=None

width=[n_x,16,8,n_y]
#### define the list of tuples (activation,derivative of activation)
activations=None
####
biases,weights= GD(x_train,y_train,x_test,width,activations,batch_size=8,learning_rate=0.05,
                num_iterations =100, print_cost=True)

### __NEEDS CHANGE__: evalutate 

Returns the number of correct predictions. Note that the output of our model is in one-hot encoding so it has 3 columns and N rows where N is the number of data points. In our MNIST exercise the test_labels __were NOT__ in one-hot encoding so they  had a single column and N rows. 

In this exercise the test_labels __ALSO__ are in one_hot_encoding

For a given row i, argmax returns the column index which has the largest value, i.e. the largest likelyhood


In [ ]:
def evaluate(test_data,test_labels,biases,weights):
    As,Z=forward_propagation(test_data,biases,weights)
    output=As[-1]
    count=0
    #the output is in one-hot encoding so it has 10 rows
    # and number of data columns where as test_tables 
    # is NOT in one-hot encoding so it has a single row
    for i in range(output.shape[0]):
        if np.argmax(output[i,:])==test_labels[i,0]:
            count=count+1
    return count/test_labels.shape[0]

In [ ]:
evaluate(x_test,y_test,biases,weights,activations)